# Building regression models for arr_delay
Identifying the most suitable model out of different modeling approaches including, linear regression, nonlinear regression, random forest regression, boosted tree regression & support verctor regression. For each model approach different hyperparameters will be analysed supported by k-fold cross-validation.

In [21]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, HuberRegressor, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from sklearn.preprocessing import QuantileTransformer
# from pprint import pprint


In [22]:
# Adjust settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
%matplotlib inline

In [23]:
# Load preprocessed dataset
data_path = '../data/processed/'
df = pd.read_pickle(os.path.join(data_path, 'final_one_hot.pkl'))

In [24]:
# Generate train/test splits
X_train_arr, X_test_arr, y_train_arr, y_test_arr = train_test_split(df_one_hot.drop(['arr_delay'], axis=1), df_one_hot['arr_delay'], test_size=0.33, random_state=42)

# Create dataframe to save model evaluation parameters
eval = pd.DataFrame(columns= ['Group', 'Model', 'R^2 test', 'RMSE test', 'R^2 train', 'RMSE train'])

## Preprocessing of data (non-linear transformation)
Using QuantileTransformer to prprocess continous data

In [25]:
# qt = QuantileTransformer(output_distribution='normal', random_state=42)
# continuous = df[['flt_tt', 'flt_sched_tt', 'sched_gt', 'act_gt', 'dep_delay',  'arr_delay']]
# df_cont = pd.DataFrame(qt.fit_transform(continuous))

# df_cont.hist()
# pyplot.show()
# Pairplot of all numerical values including dependent variable
# data = df_cont[['flt_tt', 'flt_sched_tt', 'sched_gt', 'act_gt', 'dep_delay',  'arr_delay']]
#sns.pairplot(df_cont, vars=data, diag_kind='kde')

## 1) Linear regression model

In [26]:
# Create a linear regression model for arr and dep delay
lr_arr = LinearRegression(fit_intercept=True)

# Fit models to training data
lr_arr.fit(X_train_arr, y_train_arr)

# Predict values for train and test data
lr_pred_arr_train = lr_arr.predict(X_train_arr)

lr_pred_arr_test = lr_arr.predict(X_test_arr)

# Save r^2 and RMSE for model in dataframe for later comparison
eval = eval.append({
    'Group': 'arr',
    'Model': 'Linear',
    'R^2 test': r2_score(y_test_arr, lr_pred_arr_test),
    'RMSE test': mean_squared_error(y_test_arr, lr_pred_arr_test, squared=False),
    'R^2 train': r2_score(y_train_arr, lr_pred_arr_train),
    'RMSE train': mean_squared_error(y_train_arr, lr_pred_arr_train, squared=False)
    }, ignore_index=True)

eval.round(decimals=3)

,Group,Model,R^2 test,RMSE test,R^2 train,RMSE train
0,arr,Linear,-10.418,70.135,0.914,6.165


## 2) Ridge regression

In [28]:
# Range of regularization parameter alpha
alpha = [int(x) for x in np.linspace(2, 4, 20)]

# Create random grid
param_grid = {'alpha': alpha}

# Create a ridge regression model for delay
rid_arr = Ridge(fit_intercept=True)

# Initiate the grid search models
grid_arr = GridSearchCV(estimator=rid_arr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the grid search models
grid_arr.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(grid_arr.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'alpha': 3}


In [29]:
# Create ridge regression models with best alpha values
rid_arr = Ridge(alpha=3, fit_intercept=True)

# Fit models to training data
rid_arr.fit(X_train_arr, y_train_arr)

# Predict values for train and test data
rid_pred_arr_train = rid_arr.predict(X_train_arr)

rid_pred_arr_test = rid_arr.predict(X_test_arr)

# Save r^2 and RMSE for model in dataframe for later comparison
eval = eval.append({
    'Group': 'arr',
    'Model': 'Ridge',
    'R^2 test': r2_score(y_test_arr, rid_pred_arr_test),
    'RMSE test': mean_squared_error(y_test_arr, rid_pred_arr_test, squared=False),
    'R^2 train': r2_score(y_train_arr, rid_pred_arr_train),
    'RMSE train': mean_squared_error(y_train_arr, rid_pred_arr_train, squared=False)
    }, ignore_index=True)

eval.round(decimals=3)

,Group,Model,R^2 test,RMSE test,R^2 train,RMSE train
0,arr,Linear,-10.418,70.135,0.914,6.165
1,arr,Ridge,0.895,6.741,0.912,6.227


## 3) Lasso Regression

In [31]:
# Range of regularization parameter alpha
alpha = [int(x) for x in np.linspace(0, 1, 20)]

# Create random grid
param_grid = {'alpha': alpha}

# Create a lasso regression model for arr delay
las_arr = Lasso(fit_intercept=True)

# Initiate the grid search models
grid_arr = GridSearchCV(estimator=las_arr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the grid search models
grid_arr.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(grid_arr.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'alpha': 0}


In [32]:
# Create lasso regression models with best alpha values
las_arr = Lasso(alpha=0, fit_intercept=True)

# Fit models to training data
las_arr.fit(X_train_arr, y_train_arr)

# Predict values for train and test data
las_pred_arr_train = las_arr.predict(X_train_arr)

las_pred_arr_test = las_arr.predict(X_test_arr)

# Save r^2 and RMSE for both models in dataframe for later comparison
eval = eval.append({
    'Group': 'arr',
    'Model': 'Lasso',
    'R^2 test': r2_score(y_test_arr, las_pred_arr_test),
    'RMSE test': mean_squared_error(y_test_arr, las_pred_arr_test, squared=False),
    'R^2 train': r2_score(y_train_arr, las_pred_arr_train),
    'RMSE train': mean_squared_error(y_train_arr, las_pred_arr_train, squared=False)
    }, ignore_index=True)

eval.round(decimals=3)

,Group,Model,R^2 test,RMSE test,R^2 train,RMSE train
0,arr,Linear,-10.418,70.135,0.914,6.165
1,arr,Ridge,0.895,6.741,0.912,6.227
2,arr,Lasso,0.893,6.797,0.914,6.165


## 4) Huber Regression

In [33]:
# Create the parameter grid

# Range of epsilon
epsilon = [int(x) for x in np.linspace(100, 1000, 10)]

# Range of alpha
alpha = [int(x) for x in np.linspace(0, 1, 10)]

# Create random grid
param_grid = {
    'epsilon': epsilon,
    'alpha': alpha
    }

# Create a Huber regression model for arr delay
hub_arr = HuberRegressor(fit_intercept=True)

# Initiate the grid search models
grid_arr = GridSearchCV(estimator=hub_arr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the grid search models
grid_arr.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(grid_arr.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'alpha': 1, 'epsilon': 200}


In [34]:
# Creating Huber regression model with best alpha and apsilon values
hub_arr = HuberRegressor(alpha=1, epsilon=200, fit_intercept=True)

# Fit models to training data
hub_arr.fit(X_train_arr, y_train_arr)

# Predict values for train and test data
hub_pred_arr_train = hub_arr.predict(X_train_arr)

hub_pred_arr_test = hub_arr.predict(X_test_arr)

# Save r^2 and RMSE for model in dataframe for later comparison
eval = eval.append({
    'Group': 'arr',
    'Model': 'Huber',
    'R^2 test': round(r2_score(y_test_arr, hub_pred_arr_test), 3),
    'RMSE test': round(mean_squared_error(y_test_arr, hub_pred_arr_test, squared=False), 3),
    'R^2 train': round(r2_score(y_train_arr, hub_pred_arr_train), 3),
    'RMSE train': round(mean_squared_error(y_train_arr, hub_pred_arr_train, squared=False), 3)
    }, ignore_index=True)

eval.round(decimals=3)

,Group,Model,R^2 test,RMSE test,R^2 train,RMSE train
0,arr,Linear,-10.418,70.135,0.914,6.165
1,arr,Ridge,0.895,6.741,0.912,6.227
2,arr,Lasso,0.893,6.797,0.914,6.165
3,arr,Huber,0.878,7.236,0.885,7.132


## 5) Gradient Boosting

In [54]:
# Create the parameter grid
# Range of n_estimators
n_estimators = [135, 140, 145] #[90, 100, 110], [110, 150, 200], [105, 110, 115], [115, 125, 135]

# Range of max_depth
max_depth = [12, 13, 15] #[1, 10, 20], [5, 10, 15], [8, 10, 12], [11, 12, 13]

# Random State
random_state = [42]

# Range of max_leaf_nodes
max_leaf_nodes = [14, 15, 16] #[1, 10, 20], [20, 40, 60], [15, 20, 25], [13, 15, 17]

# Create random grid
param_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'random_state': random_state,
    'max_leaf_nodes': max_leaf_nodes
    }

# Create a Huber regression model for arr delay
gb_arr = GradientBoostingRegressor()

# Initiate the grid search models
grid_arr = GridSearchCV(estimator=gb_arr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)

# Fit the grid search models
grid_arr.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(grid_arr.best_params_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'max_depth': 13, 'max_leaf_nodes': 15, 'n_estimators': 135, 'random_state': 42}


In [59]:
# Creating GradientBoosting regression model with best parameters
gb_arr = GradientBoostingRegressor(max_depth=13, max_leaf_nodes=15, n_estimators=135, random_state=42)

# Fit models to training data
gb_arr.fit(X_train_arr, y_train_arr)

# Predict values for train and test data
gb_pred_arr_train = gb_arr.predict(X_train_arr)

gb_pred_arr_test = gb_arr.predict(X_test_arr)

# Save r^2 and RMSE for model in dataframe for later comparison
eval = eval.append({
    'Group': 'arr',
    'Model': 'GradientBoosting',
    'R^2 test': round(r2_score(y_test_arr, gb_pred_arr_test), 3),
    'RMSE test': round(mean_squared_error(y_test_arr, gb_pred_arr_test, squared=False), 3),
    'R^2 train': round(r2_score(y_train_arr, gb_pred_arr_train), 3),
    'RMSE train': round(mean_squared_error(y_train_arr, gb_pred_arr_train, squared=False), 3)
    }, ignore_index=True)

eval.round(decimals=3)

,Group,Model,R^2 test,RMSE test,R^2 train,RMSE train
0,arr,Linear,-10.418,70.135,0.914,6.165
1,arr,Ridge,0.895,6.741,0.912,6.227
2,arr,Lasso,0.893,6.797,0.914,6.165
3,arr,Huber,0.878,7.236,0.885,7.132
4,arr,Decision Tree,0.860,7.772,0.890,6.979
5,arr,GradientBoosting,0.878,7.236,0.885,7.132
6,arr,GradientBoosting,0.892,6.827,0.930,5.549


## 6) ElasticNet

In [58]:
# Create the parameter grid
# Range alpha
alpha = [0, 0.1, 0.2] # [0, 1, 2], [0, 0.5, 1]

# Range of l1_ratio
l1_ratio = [0, 0.1, 0.2] # [0, 0.5, 1], [0, 0.25, 0.5]

# Range of max_depth
max_iter = [450, 500, 550] # [500, 1000, 1500], [100, 300, 500]

# Random State
random_state = [42]

# Create random grid
param_grid = {
    'alpha': alpha,
    'l1_ratio': l1_ratio,
    'random_state': random_state,
    'max_iter': max_iter
    }

# Create a Huber regression model for arr delay
en_arr = ElasticNet()

# Initiate the grid search models
grid_arr = GridSearchCV(estimator=en_arr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)

# Fit the grid search models
grid_arr.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(grid_arr.best_params_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'alpha': 0, 'l1_ratio': 0, 'max_iter': 500, 'random_state': 42}


In [60]:
# Creating ElasticNet regression model with best parameters
en_arr = ElasticNet(alpha=0, l1_ratio=0, max_iter=500, random_state=42)

# Fit models to training data
en_arr.fit(X_train_arr, y_train_arr)

# Predict values for train and test data
en_pred_arr_train = en_arr.predict(X_train_arr)

en_pred_arr_test = en_arr.predict(X_test_arr)

# Save r^2 and RMSE for model in dataframe for later comparison
eval = eval.append({
    'Group': 'arr',
    'Model': 'ElasticNet',
    'R^2 test': round(r2_score(y_test_arr, en_pred_arr_test), 3),
    'RMSE test': round(mean_squared_error(y_test_arr, en_pred_arr_test, squared=False), 3),
    'R^2 train': round(r2_score(y_train_arr, en_pred_arr_train), 3),
    'RMSE train': round(mean_squared_error(y_train_arr, en_pred_arr_train, squared=False), 3)
    }, ignore_index=True)

eval.round(decimals=3)

,Group,Model,R^2 test,RMSE test,R^2 train,RMSE train
0,arr,Linear,-10.418,70.135,0.914,6.165
1,arr,Ridge,0.895,6.741,0.912,6.227
2,arr,Lasso,0.893,6.797,0.914,6.165
3,arr,Huber,0.878,7.236,0.885,7.132
4,arr,Decision Tree,0.860,7.772,0.890,6.979
5,arr,GradientBoosting,0.878,7.236,0.885,7.132
6,arr,GradientBoosting,0.892,6.827,0.930,5.549
7,arr,ElasticNet,0.893,6.797,0.914,6.165


## 7) Decision Tree Regression

In [35]:
# Create the parameter grid to sample from during fitting

# Method of selecting samples for training each tree
ccp_alpha = [float(x) for x in np.linspace(0, 0.2, num=11)]
# Measurement of the quality of a split
criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
# Maximum number of leafs in tree
max_depth = [int(x) for x in np.linspace(80, 100, num=11)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum leaf nodes
max_leaf_nodes = [int(x) for x in np.linspace(140, 180, num=11)]
# Minimum impurity decrease
min_impurity_decrease = [float(x) for x in np.linspace(0.1, 0.3, num=11)]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(5, 15, num=11)]
# Minimum weightes fraction of the sum of total of weights required to be at a leaf node
min_weight_fraction_leaf = [float(x) for x in np.linspace(0, 0.1, num=11)]
# Strategy used to split at each node.
splitter = ['best', 'random']

# Create the random grid
random_grid = {
    'ccp_alpha': ccp_alpha,
    'criterion': criterion,
    'max_depth': max_depth,
    'max_features': max_features,
    'max_leaf_nodes': max_leaf_nodes,
    'min_impurity_decrease': min_impurity_decrease,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'min_weight_fraction_leaf': min_weight_fraction_leaf,
    'splitter': splitter
    }

# Create a decision tree regression model for arr delay
dt_arr = DecisionTreeRegressor()

# Initiate the grid search models
dt_arr_random = RandomizedSearchCV(estimator=dt_arr, param_distributions=random_grid, n_iter=100, cv=5, n_jobs=-1, random_state=42, verbose=1)

# Fit the grid search models
dt_arr_random.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(dt_arr_random.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'splitter': 'best', 'min_weight_fraction_leaf': 0.0, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0.18, 'max_leaf_nodes': 176, 'max_features': 'auto', 'max_depth': 90, 'criterion': 'friedman_mse', 'ccp_alpha': 0.18}


In [36]:
""" # Create the parameter grid to sample from during fitting

# Method of selecting samples for training each tree
ccp_alpha = [0.18]
# Measurement of the quality of a split
criterion = ['friedman_mse']
# Maximum number of leafs in tree
max_depth = [90]
# Number of features to consider at every split
max_features = ['auto']
# Maximum leaf nodes
max_leaf_nodes = [176]
# Minimum impurity decrease
min_impurity_decrease = [0.18]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Minimum number of samples required to split a node
min_samples_split = [5]
# Minimum weightes fraction of the sum of total of weights required to be at a leaf node
min_weight_fraction_leaf = [0]
# Strategy used to split at each node.
splitter = ['best']

# Create the random grid
param_grid = {
    'ccp_alpha': ccp_alpha,
    'criterion': criterion,
    'max_depth': max_depth,
    'max_features': max_features,
    'max_leaf_nodes': max_leaf_nodes,
    'min_impurity_decrease': min_impurity_decrease,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'min_weight_fraction_leaf': min_weight_fraction_leaf,
    'splitter': splitter
    }

# Create a decision tree regression model for arr and dep delay
dt_arr = DecisionTreeRegressor()

# Initiate the grid search models
dt_arr_grid = GridSearchCV(estimator=dt_arr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the grid search models
dt_arr_grid.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(dt_arr_grid.best_params_) """

Fitting 5 folds for each of 1 candidates, totalling 5 fits
{'ccp_alpha': 0.18, 'criterion': 'friedman_mse', 'max_depth': 90, 'max_features': 'auto', 'max_leaf_nodes': 176, 'min_impurity_decrease': 0.18, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0, 'splitter': 'best'}


In [38]:
# Creating Decision Tree regression models with best parameters
dt_arr = DecisionTreeRegressor(ccp_alpha=0.18, criterion='friedman_mse', max_depth=90, max_features='auto',
    max_leaf_nodes=176, min_impurity_decrease=0.18, min_samples_leaf=1, min_samples_split=5, min_weight_fraction_leaf=0, splitter='best')

# Fit models to training data
dt_arr.fit(X_train_arr, y_train_arr)

# Predict values for train and test data
dt_pred_arr_train = dt_arr.predict(X_train_arr)

dt_pred_arr_test = dt_arr.predict(X_test_arr)

# Save r^2 and RMSE for model in dataframe for later comparison
eval = eval.append({
    'Group': 'arr',
    'Model': 'Decision Tree',
    'R^2 test': round(r2_score(y_test_arr, dt_pred_arr_test), 3),
    'RMSE test': round(mean_squared_error(y_test_arr, dt_pred_arr_test, squared=False), 3),
    'R^2 train': round(r2_score(y_train_arr, dt_pred_arr_train), 3),
    'RMSE train': round(mean_squared_error(y_train_arr, dt_pred_arr_train, squared=False), 3)
    }, ignore_index=True)

eval.round(decimals=3)

,Group,Model,R^2 test,RMSE test,R^2 train,RMSE train
0,arr,Linear,-10.418,70.135,0.914,6.165
1,arr,Ridge,0.895,6.741,0.912,6.227
2,arr,Lasso,0.893,6.797,0.914,6.165
3,arr,Huber,0.878,7.236,0.885,7.132
4,arr,Decision Tree,0.860,7.772,0.890,6.979


## 8) Random Forest

In [39]:
# Create the parameter grid to sample from during fitting

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of leafs in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
    }

# Create a decision tree regression model for arr delay
rf_arr = RandomForestRegressor()

# Initiate the grid search models
rf_arr_random = RandomizedSearchCV(estimator=rf_arr, param_distributions=random_grid, n_iter=100, cv=5, n_jobs=-1, random_state=42, verbose=1)

# Fit the grid search models
rf_arr_random.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(rf_arr_random.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

## 9) Support Vector Regression

In [63]:
# Create the parameter grid
# Range alpha
kernel = ['linear', 'poly', 'rbf', 'sigmoid']

# Create random grid
param_grid = {
    'kernel': kernel
    }

# Create a Huber regression model for arr delay
svr_arr = SVR()

# Initiate the grid search models
grid_arr = GridSearchCV(estimator=svr_arr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)

# Fit the grid search models
grid_arr.fit(X_train_arr, y_train_arr)

# Print best parameters for the models
print(grid_arr.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'kernel': 'linear'}


In [64]:
# Creating SVR regression model with best parameters
svr_arr = SVR(kernel='linear')

# Fit models to training data
svr_arr.fit(X_train_arr, y_train_arr)

# Predict values for train and test data
svr_pred_arr_train = svr_arr.predict(X_train_arr)

svr_pred_arr_test = svr_arr.predict(X_test_arr)

# Save r^2 and RMSE for model in dataframe for later comparison
eval = eval.append({
    'Group': 'arr',
    'Model': 'SVR',
    'R^2 test': round(r2_score(y_test_arr, svr_pred_arr_test), 3),
    'RMSE test': round(mean_squared_error(y_test_arr, svr_pred_arr_test, squared=False), 3),
    'R^2 train': round(r2_score(y_train_arr, svr_pred_arr_train), 3),
    'RMSE train': round(mean_squared_error(y_train_arr, svr_pred_arr_train, squared=False), 3)
    }, ignore_index=True)

eval.round(decimals=3)

,Group,Model,R^2 test,RMSE test,R^2 train,RMSE train
0,arr,Linear,-10.418,70.135,0.914,6.165
1,arr,Ridge,0.895,6.741,0.912,6.227
2,arr,Lasso,0.893,6.797,0.914,6.165
3,arr,Huber,0.878,7.236,0.885,7.132
4,arr,Decision Tree,0.860,7.772,0.890,6.979
5,arr,GradientBoosting,0.878,7.236,0.885,7.132
6,arr,GradientBoosting,0.892,6.827,0.930,5.549
7,arr,ElasticNet,0.893,6.797,0.914,6.165
8,arr,SVR,0.891,6.837,0.907,6.432


Confidence over the day:
RMSE in Abhängigkeit der Flüge des Tages
RMSE in Abhängigkeit der Uhrzeit


PCA
Lineare Regression Drop der Variablen

## Finding the best overall model

In [ ]:
# Initializing the different estimators to be tested
est1 = LinearRegression()
est2 = Ridge()
est3 = Lasso()
est4 = HuberRegressor()
est5 = GradientBoostingRegressor()
est6 = ElasticNet()
est7 = DecisionTreeRegressor()
est8 = RandomForestRegressor()
est9 = SVR()